In [7]:
!pip install datasets

In [2]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))


True
1
Tesla T4


In [3]:
import nltk
from nltk.corpus import wordnet as wn
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from datasets import Dataset

nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [4]:
def create_wordnet_antonym_dataset():
    antonym_pairs = []

    for syn in wn.all_synsets():
        for lemma in syn.lemmas():
            if lemma.antonyms():
                word = lemma.name().replace('_', ' ')
                antonym = lemma.antonyms()[0].name().replace('_', ' ')
                antonym_pairs.append((word, antonym))
    df = pd.DataFrame(antonym_pairs, columns=['input_phrase', 'opposite_phrase'])
    return df
df = create_wordnet_antonym_dataset()


In [5]:
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=1,
    save_steps=10_000,
    logging_steps=500,
    do_eval=True,
    fp16=True
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

def preprocess_function(examples):
    inputs = [f"find opposite: {phrase}" for phrase in examples["input_phrase"]]
    model_inputs = tokenizer(inputs, max_length=32, truncation=True, padding="max_length")
    labels = tokenizer(text_target=examples["opposite_phrase"], max_length=32, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(preprocess_function, batched=True)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    data_collator=data_collator,
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map:   0%|          | 0/7768 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,0.260500,0.204012
2,0.223300,0.185003
3,0.217600,0.180780


TrainOutput(global_step=5826, training_loss=0.377160124380974, metrics={'train_runtime': 706.8764, 'train_samples_per_second': 32.968, 'train_steps_per_second': 8.242, 'total_flos': 197125333843968.0, 'train_loss': 0.377160124380974, 'epoch': 3.0})

In [6]:
model.save_pretrained("./fine-tuned-model")
tokenizer.save_pretrained("./fine-tuned-tokenizer")


('./fine-tuned-tokenizer/tokenizer_config.json',
 './fine-tuned-tokenizer/special_tokens_map.json',
 './fine-tuned-tokenizer/spiece.model',
 './fine-tuned-tokenizer/added_tokens.json')